Train face recognition model

In [40]:
import csv
import os
import pickle
from pathlib import Path
from typing import List, Tuple, Optional, Iterable

import face_recognition
import numpy as np
from sklearn.svm import SVC


In [41]:
# =========================
# CẤU HÌNH ĐƯỜNG DẪN
# =========================
ROOT_DIR = os.getcwd()
DATA_DIR = os.path.join(ROOT_DIR, "data")
LFW_DIR = os.path.join(DATA_DIR, "lfw-deepfunneled", "lfw-deepfunneled")

TRAIN_CSV = os.path.join(DATA_DIR, "peopleDevTrain.csv")
TEST_CSV = os.path.join(DATA_DIR, "peopleDevTest.csv")

MODEL_PATH = os.path.join(ROOT_DIR, "models", "face_svc.pkl")
ENC_CACHE = os.path.join(ROOT_DIR, "models", "train_encodings.pkl")

DEFAULT_TEST_DIR = os.path.join(ROOT_DIR, "test")
DEFAULT_TEST_OUT = os.path.join(ROOT_DIR, "test", "test_results.csv")
DEFAULT_EVAL_DIR = os.path.join(ROOT_DIR, "test_100_peoples")

In [42]:
# =========================
# CẤU HÌNH FACE PIPELINE
# =========================
DETECT_MODEL = "cnn"         # "hog" hoặc "cnn"
LANDMARKS_MODEL = "large"    # "small" hoặc "large"
UPSAMPLE = 0                # 0 nhanh, tăng lên (1,2) bắt mặt nhỏ tốt hơn nhưng chậm

In [43]:
# =========================
# ĐỌC DANH SÁCH DỮ LIỆU
# =========================
def read_people(csv_path: Path) -> List[Tuple[str, int]]:
    """Đọc danh sách người và số lượng ảnh cần dùng từ file CSV."""
    people = []
    with open(csv_path, newline="", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            name = row["name"]
            count = int(row["images"])
            people.append((name, count))
    return people


def image_paths_for_person(name: str, count: int) -> List[str]:
    """Sinh danh sách đường dẫn ảnh cho một người theo định dạng LFW."""
    paths = []
    for idx in range(1, count + 1):
        filename = f"{name}_{idx:04d}.jpg"
        paths.append(os.path.join(LFW_DIR, name, filename))
    return paths

In [44]:
# =========================
# TIỀN XỬ LÝ ẢNH / ENCODING
# =========================
def _box_area(box: Tuple[int, int, int, int]) -> int:
    """Tính diện tích box (top, right, bottom, left)."""
    top, right, bottom, left = box
    return max(0, bottom - top) * max(0, right - left)


def encode_image(
    image_path: str,
    detect_model: str = DETECT_MODEL,
    landmarks_model: str = LANDMARKS_MODEL,
    upsample: int = UPSAMPLE,
    choose_largest_face: bool = True
) -> Optional[np.ndarray]:
    """
    Đọc ảnh và trả về encoding đầu tiên (hoặc mặt lớn nhất nếu choose_largest_face=True).

    Pipeline:
    - Kiểm tra file
    - Load ảnh
    - Detect face locations (hog/cnn)
    - (Tuỳ chọn) chọn 1 mặt lớn nhất nếu có nhiều mặt
    - Face encodings (embedding 128D) với landmarks model (small/large)
    """
    if not os.path.isfile(image_path):
        print(f"[BỎ QUA] Không tìm thấy ảnh: {image_path}")
        return None

    try:
        image = face_recognition.load_image_file(image_path)
    except Exception as e:
        print(f"[BỎ QUA] Lỗi đọc ảnh: {image_path} | {e}")
        return None

    # 1) Detect mặt (bounding boxes)
    try:
        locs = face_recognition.face_locations(
            image,
            number_of_times_to_upsample=upsample,
            model=detect_model
        )
    except Exception as e:
        print(f"[BỎ QUA] Lỗi detect face: {image_path} | {e}")
        return None

    if not locs:
        print(f"[BỎ QUA] Không tìm thấy khuôn mặt trong ảnh: {image_path}")
        return None

    # 2) Nếu có nhiều mặt: chọn mặt lớn nhất (thường là chủ thể)
    if choose_largest_face and len(locs) > 1:
        locs = [max(locs, key=_box_area)]

    # 3) Tạo embedding 128D dựa trên locations đã biết
    try:
        encs = face_recognition.face_encodings(
            image,
            known_face_locations=locs,
            model=landmarks_model  # "small" hoặc "large"
        )
    except Exception as e:
        print(f"[BỎ QUA] Lỗi tạo encodings: {image_path} | {e}")
        return None

    if not encs:
        print(f"[BỎ QUA] Không tạo được encoding cho ảnh: {image_path}")
        return None

    return encs[0]


def build_dataset(csv_path: Path):
    """Từ file CSV sinh X (encodings) và y (nhãn)."""
    people = read_people(csv_path)
    X, y = [], []
    for name, count in people:
        for img_path in image_paths_for_person(name, count):
            enc = encode_image(img_path)
            if enc is None:
                continue
            X.append(enc)
            y.append(name)

    if not X:
        raise RuntimeError(f"Không tạo được dữ liệu từ {csv_path}")

    return np.vstack(X), np.array(y)

In [45]:
# =========================
# HUẤN LUYỆN MODEL
# =========================
def train(train_csv: Path = TRAIN_CSV, model_path: Path = MODEL_PATH):
    """Huấn luyện SVC và lưu model."""
    print(f"[TRAIN] Đang xây dựng tập train từ {train_csv}")
    X_train, y_train = build_dataset(train_csv)
    print(f"[TRAIN] Tổng mẫu train: {len(y_train)}")

    clf = SVC(kernel="linear", probability=True, class_weight="balanced")
    clf.fit(X_train, y_train)
    print("[TRAIN] Huấn luyện xong.")

    model_path.parent.mkdir(parents=True, exist_ok=True)
    with open(model_path, "wb") as f:
        pickle.dump(clf, f)
    print(f"[TRAIN] Đã lưu model vào {model_path}")

    # Lưu cache encodings để predict_dist khỏi phải build lại lâu
    ENC_CACHE.parent.mkdir(parents=True, exist_ok=True)
    with open(ENC_CACHE, "wb") as f:
        pickle.dump({"X_train": X_train, "y_train": y_train}, f)
    print(f"[TRAIN] Đã lưu cache encodings vào {ENC_CACHE}")

In [ ]:
train()

[TRAIN] Đang xây dựng tập train từ C:\Users\PC-09\Documents\Projects\face_recognition_svm\data\peopleDevTrain.csv
